In [6]:
import ROOT as rt
import csv
import re
import sys
import collections

from collections import OrderedDict
import uproot
from root_numpy import array2tree


import scipy
import awkward
import numpy as np
import time
import numba
from numba import jit
from matplotlib import pyplot as plt
import os
sys.path.append(os.getenv('HOME')+'/gpu/christiw/llp/delayed_jet_analyzer/lib/')
from histo_utilities import create_TH1D, create_TH2D, std_color_list, create_TGraph, make_ratio_plot
from helper_functions import deltaPhi
import numpy as np
from scipy.stats import norm
import math
import CMS_lumi, tdrstyle
a= tdrstyle.setTDRStyle()
CMS_lumi.writeExtraText = 0

wH = 1
Z_MASS = 91.2



# donotdelete = []
print(sys.version)

3.6.8 (default, Aug  7 2019, 17:28:10) 
[GCC 4.8.5 20150623 (Red Hat 4.8.5-39)]


# Load ntuples

In [3]:
fpath =OrderedDict()
tree = OrderedDict()

data_year = 'full'
lumi = {'MC_Summer16':35.9*1000,
        'MC_Fall17': 41.5*1000, 
        'MC_Fall18':  59.7*1000 ,
       
       }


path = '/storage/af/group/phys_exotica/delayedjets/HNL/skim/'



# fpath['data'] = path+ 'EGamma_2018A.root'
# fpath['wjets'] = path+ 'WJetsToLNu_1pb_weighted.root'
sample = ['HNL_electronType_mHNL10p0_pl10000',
    'HNL_electronType_mHNL10p0_pl1000',
    'HNL_electronType_mHNL10p0_pl100',
    'HNL_electronType_mHNL1p0_pl1000',
    'HNL_electronType_mHNL1p0_pl100',
    'HNL_electronType_mHNL1p0_pl10',
    'HNL_electronType_mHNL2p0_pl1000',
    'HNL_electronType_mHNL2p0_pl100',
    'HNL_electronType_mHNL2p0_pl10',
    'HNL_electronType_mHNL4p0_pl1000',
    'HNL_electronType_mHNL4p0_pl100',
    'HNL_electronType_mHNL4p0_pl10',
    'HNL_electronType_mHNL7p0_pl10000',
    'HNL_electronType_mHNL7p0_pl1000',
    'HNL_electronType_mHNL7p0_pl100',
        ]

for s in sample:
    fpath[s] = path + s + '_1pb_weighted.root'
NEvents = {}
Total = {}
accep = {}
accep_met = {}
for k,v in fpath.items():
    print (k, v)
    root_dir = uproot.open(v) 
#     if not b'MuonSystem;1' in root_dir.keys():print("ERROR")
#     else:
    tree[k] = root_dir['MuonSystem']
    NEvents[k] = root_dir['NEvents'][1]
    a = tree[k]["weight"].array()
    print("NEvents",NEvents[k],np.sum(a))


HNL_electronType_mHNL10p0_pl10000 /storage/af/group/phys_exotica/delayedjets/HNL/skim/HNL_electronType_mHNL10p0_pl10000_1pb_weighted.root
NEvents 8.062509 1.2099996e-05
HNL_electronType_mHNL10p0_pl1000 /storage/af/group/phys_exotica/delayedjets/HNL/skim/HNL_electronType_mHNL10p0_pl1000_1pb_weighted.root
NEvents 80.75944 0.00012100019
HNL_electronType_mHNL10p0_pl100 /storage/af/group/phys_exotica/delayedjets/HNL/skim/HNL_electronType_mHNL10p0_pl100_1pb_weighted.root
NEvents 805.5457 0.001210002
HNL_electronType_mHNL1p0_pl1000 /storage/af/group/phys_exotica/delayedjets/HNL/skim/HNL_electronType_mHNL1p0_pl1000_1pb_weighted.root
NEvents 9076984.0 13.569979
HNL_electronType_mHNL1p0_pl100 /storage/af/group/phys_exotica/delayedjets/HNL/skim/HNL_electronType_mHNL1p0_pl100_1pb_weighted.root
NEvents 90420920.0 135.70013
HNL_electronType_mHNL1p0_pl10 /storage/af/group/phys_exotica/delayedjets/HNL/skim/HNL_electronType_mHNL1p0_pl10_1pb_weighted.root
NEvents 907944640.0 1356.9989
HNL_electronType_m

In [7]:
# make Wpt
path = os.getenv('HOME')+'/login-1/christiw/LLP/CMSSW_9_4_4/src/llp_analyzer/data/WPtWeights/'
target = uproot.open(path + 'wp-13tev-cms.root')['s_qt']
source = uproot.open(path + 'WPT.root')['Wpt']
  


# nCsc with different hit vetoing

In [ ]:

sel_ev = {}
weight = {}

dphiMetXYCorr_cluster = {}

dtRechitCluster_match_RPCBx_dPhi0p5_mode = {}
dtRechitClusterPhi = {}
dtRechitClusterNStation10 = {}
dtRechitClusterEta = {}
dtRechitClusterPhi = {}
dtRechitClusterAvgStation10 = {}
dtRechitClusterMaxStation = {}
dphiMet_cluster = {}
dtRechitCluster_match_RPCBx_dR0p4_mode = {}
dtClusterSize = {}
dtRechitClusterMuonVetoPt = {}
dtRechitClusterJetVetoPt = {}
dphiLep_cluster = {}
dtRechitCluster_match_RPCBx_dPhi0p5 = {}
dtRechitCluster_match_RB1_0p4 = {}
dtRechitCluster_match_MB1hits_0p5 = {}
dtRechitCluster_match_RPChits_dPhi0p5 = {}
weight = {}
weight_evt = {}
gWPt = {}

for k,T in tree.items():
########### SELECTION: CLUSTERS ############


    sel_rechitcluster = np.logical_and( T.array('dtRechitClusterMuonVetoPt') < 10, T.array('dtRechitClusterJetVetoPt') < 20)
     
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('dtRechitCluster_match_RPChits_dPhi0p5') > 0)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('dtRechitCluster_match_MB1hits_cosmics_minus') <= 8)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('dtRechitCluster_match_MB1hits_cosmics_plus') <= 8)

  
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('dtRechitCluster_match_MB1hits_0p5') <= 1)
    if 'HNL' in k:
        sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('dtRechitCluster_match_gLLP'))
        sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('dtRechitCluster_match_gLLP_dt'))


    
########### SELECTION: LEPTON ############
    
    sel_ele = np.logical_and(T.array('lepPt') > 35, np.abs(T.array('lepEta'))<2.4)
    sel_ele = np.logical_and(sel_ele, np.abs(T.array('lepPdgId')) == 11)
    sel_ele = np.logical_and(sel_ele, np.abs(T.array('lepPassId')) == 1)
    
########### SELECTION: EVENTS ############

    sel_ev[k] = T.array('SingleEleTrigger')
    sel_ev[k] = np.logical_and(sel_ev[k] ,sel_ele.sum()>=1)
    sel_ev[k] = np.logical_and(sel_ev[k] ,T.array('metEENoise')>=30)
    
    sel_ev[k] = np.logical_and(sel_ev[k],T.array('Flag2_all'))

#     sel_ev[k] = np.logical_and(sel_ev[k],T.array('nDtStations25')<3)
#     sel_ev[k] = np.logical_and(sel_ev[k],T.array('nDtWheels25')<3)
    sel_ev[k]  = np.logical_and(sel_ev[k],sel_rechitcluster.sum() >= 1)

#     sel_ev[k] = np.logical_and(sel_ev[k],np.abs(T.array('jetMet_dPhiMin'))>0.6)
#     sel_ev[k] = np.sum(T.array('gLLP_dt'),axis = 1) >= 1
    
########### BRANCHES ############
    
    dtRechitClusterNStation10[k] = T.array('dtRechitClusterNStation10')[sel_rechitcluster][sel_ev[k]][:,0].flatten()
    dtRechitClusterEta[k] = T.array('dtRechitClusterEta')[sel_rechitcluster][sel_ev[k]][:,0].flatten()
    dtRechitClusterPhi[k] = T.array('dtRechitClusterPhi')[sel_rechitcluster][sel_ev[k]][:,0].flatten()
    dtRechitClusterAvgStation10[k] = T.array('dtRechitClusterAvgStation10')[sel_rechitcluster][sel_ev[k]][:,0].flatten()
    dtRechitClusterMaxStation[k] = T.array('dtRechitClusterMaxStation')[sel_rechitcluster][sel_ev[k]][:,0].flatten()
    dphiMet_cluster[k] = T.array('dtRechitClusterMetEENoise_dPhi')[sel_rechitcluster][sel_ev[k]][:,0].flatten()
    dtClusterSize[k] =  T.array('dtRechitClusterSize')[sel_rechitcluster][sel_ev[k]][:,0].flatten()
    
    
    dtRechitClusterMuonVetoPt[k] = T.array('dtRechitClusterMuonVetoPt')[sel_rechitcluster][sel_ev[k]][:,0].flatten()
    dtRechitClusterJetVetoPt[k] = T.array('dtRechitClusterJetVetoPt')[sel_rechitcluster][sel_ev[k]][:,0].flatten()
    dphiLep_cluster[k] = deltaPhi(T.array('lepPhi')[sel_ele][sel_ev[k]][:,0], dtRechitClusterPhi[k])

    
    dtRechitCluster_match_RPCBx_dPhi0p5[k] =  T.array('dtRechitCluster_match_RPCBx_dPhi0p5')[sel_rechitcluster][sel_ev[k]][:,0].flatten()
    dtRechitCluster_match_RB1_0p4[k] =  T.array('dtRechitCluster_match_RB1_0p4')[sel_rechitcluster][sel_ev[k]][:,0].flatten()
    dtRechitCluster_match_MB1hits_0p5[k] =  T.array('dtRechitCluster_match_MB1hits_0p5')[sel_rechitcluster][sel_ev[k]][:,0].flatten()
    dtRechitCluster_match_RPChits_dPhi0p5[k] =  T.array('dtRechitCluster_match_RPChits_dPhi0p5')[sel_rechitcluster][sel_ev[k]][:,0].flatten()
    gWPt[k] = T.array('gWPt')
    if 'data' in k:
        weight[k] = np.ones(dphiLep_cluster[k].shape)
    else:
        weight[k] = T.array('weight')*T.array('pileupWeight')
        weight[k] *= target.values[np.argmax(target.edges>gWPt[k][:,None],axis=1)-1]/11178549.542718 # integral
        weight[k] /= source.values[np.argmax(source.edges>gWPt[k][:,None],axis=1)-1]
        weight[k] = weight[k][sel_ev[k]]
        print(k, '\t', np.sum(weight[k])*137000)

HNL_electronType_mHNL10p0_pl10000 	 0.0011271130500034587
HNL_electronType_mHNL10p0_pl1000 	 0.0272698645460423
HNL_electronType_mHNL10p0_pl100 	 0.0007543376496776943
HNL_electronType_mHNL1p0_pl1000 	 1043.5002953745425
HNL_electronType_mHNL1p0_pl100 	 24833.994537591934
HNL_electronType_mHNL1p0_pl10 	 508.7742190808058
HNL_electronType_mHNL2p0_pl1000 	 51.73762657796033
HNL_electronType_mHNL2p0_pl100 	 378.1738199759275
HNL_electronType_mHNL2p0_pl10 	 0.0
HNL_electronType_mHNL4p0_pl1000 	 2.8105623459850904
HNL_electronType_mHNL4p0_pl100 	 2.181016436225036


# signal yield

In [ ]:

BR = 0.01
DPHI_CUT = 1
var = dphiMet_cluster

for N_RECHIT_CUT in np.arange(60,220,10):
    if not N_RECHIT_CUT==100:continue
        
    for m in mass:
        signal_rate = []
        signal_unc = []
        for ct in OLD_CTAU:   
            k = 'MC_'+str(m)+'_'+str(ct)                       
            cond = np.logical_and(dtClusterSize[k]>=N_RECHIT_CUT, np.abs(var[k])<DPHI_CUT)#bin D
            signal_rate.append(np.sum(weight[k][cond]))
            signal_unc.append(np.sum(weight[k][cond]*weight[k][cond])**0.5)
        signal_rate = np.array(signal_rate)
        signal_unc = np.array(signal_unc)
        
        print(m, '\t', '\t'.join(map(str, [round(num,1) for num in signal_rate*BR])))
